In [5]:
import tvb
%pylab nbagg
from tvb.simulator.lab import *
LOG= get_logger('demo')

from scipy.io import loadmat,savemat

import glob

import pandas as pd

import numpy as np
import math
import openpyxl as px

# Loading in an excel matrix

Locate your excel file  and load it into a workbook.

In [6]:
f = '/home/htian/Downloads/FV91_bilateral_20130226_AddedRefs.xlsx'

In [7]:
w= px.load_workbook(f)

In [8]:
w.get_sheet_names()

[u'FV91_bilateral_withA2_ORIG',
 u'FV91_wAddedRefs(4)',
 u'Refs for Added Connections',
 u'ForTVBsims']

Get the sheet you want to load in.

In [7]:
p= w.get_sheet_by_name(name='FV91_wAddedRefs(4)')

Optional: load your sheet into a DataFrame to view sheet. 

In [8]:
df= pd.DataFrame(p.values)
df

,0,1,2,3,4,5,6,7,8,9,...,145,146,147,148,149,150,151,152,153,154
0,None,FV91-1_L,BP89-10_L,BP89-11_L,BP89-12_L,BP89-13_L,BP89-14_L,FV91-2_L,IAC87a-23_L,IAC87a-24_L,...,FV91-TH_R,FV91-V1_R,FV91-V2_R,FV91-V3_R,FV91-V3A_R,FV91-V4_R,FV91-V4t_R,FV91-VIP_R,FV91-VOT_R,FV91-VP_R
1,FV91-1_L,2,-1,-1,-1,-1,0,2,2,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,BP89-10_L,-1,3,3,3,3,3,2,2,3,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,BP89-11_L,-1,3,3,3,3,3,2,2,3,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,BP89-12_L,-1,3,3,3,3,3,2,2,3,...,3,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,BP89-13_L,-1,3,3,3,3,3,2,2,3,...,2,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,BP89-14_L,0,3,3,3,3,3,0,2,3,...,2,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,FV91-2_L,2,-1,-1,-1,-1,0,2,2,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,IAC87a-23_L,2,2,2,2,0,0,2,2,3,...,0,0,3,-1,-1,-1,-1,-1,0,-1
9,IAC87a-24_L,0,3,3,3,3,3,2,2,3,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1


Append values into a matrix by looping through columns and rows. 

In [14]:
r= []
for i in range(155):
    r.append(i +1)

In [15]:
a= []
for row in r:
    for col in r:
        a.append(p.cell(column= col, row= row).value)
a= np.array(a)

Resize your matrix to the right size and remove the labels in index 0. Edit your values to only include the numerical weighting.

In [16]:
mtx= np.resize(a, [155,155])
m= mtx[1:155, 1:155]

In [17]:
for i in range(154):
    for j in range(154):
        if m[i,j]== -1L:
            m[i,j]= -1
        elif m[i,j]==  0L:
            m[i,j]= 0
        elif m[i,j]==1L:
            m[i,j]= 1
        elif m[i,j] == 2L:
            m[i,j]= 2
        elif m[i,j]==3L:
            m[i,j]= 3
        elif m[i,j]==4L:
            m[i,j]= 4
        else:
            print m[i,j]

---

# Making a connectivity zip file for TVB from a Matlab file

#### 1. Make the weights file

Locate your file and load it in then load in the weights. We want to normalize the values so take the max of the weights. Loop through each element and divide it by the max. We are also going to log the values, and change any negative infinity values to 0. Finally, see where you want to save the file and save it as a text file. 

In [18]:
cd /home/kshen/forHeather/

In [19]:
F = loadmat('FV91_avg9_WeightsDistances.mat')

In [20]:
weights = F['avgWeight']

In [21]:
weights

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.59887527e-01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.59887527e-01,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   8.15789694e-05],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   7.62142806e-02],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          8.15789694e-05,   7.62142806e-02,   0.00000000e+00]])

In [22]:
np.unique(weights)

array([  0.00000000e+00,   5.18466257e-06,   1.04676743e-05, ...,
         4.06721765e-01,   4.54920578e-01,   4.79999733e-01])

In [23]:
weights.max()

0.47999973264005447

In [24]:
weights_norm= weights.copy()

In [25]:
for i in range(len(weights_norm)):
    for j in range(len(weights_norm)):
        weights_norm[i,j] /= weights.max()
np.unique(weights_norm)

array([  0.00000000e+00,   1.08013864e-05,   2.18076668e-05, ...,
         8.47337482e-01,   9.47751731e-01,   1.00000000e+00])

In [26]:
log(weights_norm[0])
for i in range(len(weights_norm)):
    for j in range(len(weights_norm)):
        if weights_norm[i,j] == -inf:
            weights_norm[i,j] = 0

In [27]:
new_mtx= []
for i in range(154):
    for j in range(154):
        if m[i,j] ==0 or m[i,j] == -1:
            new_mtx.append(0)
        else:
            new_mtx.append(weights_norm[i,j])
new_mtx= np.array(np.reshape(new_mtx,[154,154]))

In [28]:
for i in range(len(new_mtx)): 
    for j in range(len(new_mtx)): 
        new_mtx[i,j] *= 3


In [29]:
cd /home/htian/Data/

In [30]:
np.savetxt('avg9/weights.txt', new_mtx)

#### 2. Make the tract lengths file

Just like how you loaded in the weights, load in the distance matrix. Our matrix had 'nan' values so we will use the nan to num function to convert them to numbers. Then go ahead and normalize the values just like we just did. Then just save your file as a text file again. 

In [31]:
tls = F['avgDistance']

In [32]:
np.unique(tls)

array([ 0.       ,  2.9613557,  3.0847721, ...,        nan,        nan,
              nan])

In [33]:
tls[2,6]

nan

In [34]:
tls_nonan= np.nan_to_num(tls)

In [35]:
np.unique(tls_nonan)

array([  0.        ,   2.9613557 ,   3.0847721 , ...,  77.91365772,
        78.13042662,  83.92245356])

In [37]:
tls_nonan.max()

83.922453562418625

In [38]:
tls_norm= tls_nonan.copy()

In [39]:
for i in range(len(tls_norm)):
    for j in range(len(tls_norm)):
        tls_norm[i,j] /= tls_nonan.max()
        
np.unique(tls_norm)

array([ 0.        ,  0.03528681,  0.03675741, ...,  0.92840062,
        0.93098358,  1.        ])

In [40]:
np.savetxt('avg9/tract_lengths.txt', tls_norm)

***4. Zip the files together***

Finally, zip the files you want together. 

In [41]:
!zip -rj avg9.zip avg9